In [ ]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np
from time import sleep

import time
from RobotClass import Robot

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

import ipywidgets.widgets as widgets
from IPython.display import display, HTML

In [ ]:
class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

In [ ]:
# Function to find the position of the line in the image
def find_line_position(image, lower_color_range, upper_color_range):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Create a mask for the specified color range
    mask = cv2.inRange(hsv, lower_color_range, upper_color_range)
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    line_position = None
    max_area = 0
    
    # Find the contour with the largest area
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if area > max_area:
            max_area = area
            # Calculate the centroid of the contour
            moments = cv2.moments(contour)
            if moments['m00'] != 0:
                cx = int(moments['m10'] / moments['m00'])
                line_position = cx
    
    return line_position

In [ ]:
# Function to make the robot follow the line
def follow_line(robot, line_position, image_center, kp=0.01, speed=0.2):
    if line_position is not None:
        # Calculate the error between the line position and the image center
        error = image_center - line_position
        # Calculate the turn speed based on the error
        turn_speed = error * kp
        # Calculate the speed for each wheel
        left_speed = speed - turn_speed
        right_speed = speed + turn_speed
        # Set the motor speeds
        robot.set_motors(left_speed, right_speed, left_speed, right_speed)
    else:
        robot.stop()

In [ ]:
def update_image_widget(image):
    _, encoded_image = cv2.imencode('.jpg', image)
    image_widget.value = encoded_image.tobytes()

In [ ]:
# Initialize the robot
robot = Robot()

# Initialize the camera
camera = Camera.instance()

# Set the color range for the line (yellow or blue)
lower_color_range = np.array([20, 100, 100])  # Modify these values for the line color
upper_color_range = np.array([30, 255, 255])  # Modify these values for the line color

# Create an image widget for displaying the camera feed
image_widget = widgets.Image(format='jpeg', width=300, height=300)
display(image_widget)

try:
    while True:
        # Capture an image
        image = camera.color_value

        # Update the image widget with the current camera frame
        update_image_widget(image)

        # Find the line position
        line_position = find_line_position(image, lower_color_range, upper_color_range)

        # Make the robot follow the line
        image_center = image.shape[1] // 2
        follow_line(robot, line_position, image_center)

        # Sleep for a while
        sleep(0.1)

except KeyboardInterrupt:
    pass
finally:
    robot.stop()